# importing necessary libraries

In [28]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# getting data from internet

In [37]:
web_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_text= requests.get(web_link).text


# using beautiful soup to parse the HTML/XML codes

In [40]:
Canada_data = BeautifulSoup(web_text, 'lxml')

<b>Processing-part-1: extracting raw table (from webpage)<b>

# extracting the raw table inside that webpage

In [42]:
table_columns = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = table_columns)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
        i = 0
        for td in tr.find_all('td'):
            if i == 0:
                postcode = td.text
                i = i + 1
            elif i == 1:
                borough = td.text
                i = i + 1
            elif i == 2: 
                neighborhood = td.text.strip('\\n').replace(']','')
        toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# Ignore cells with a borough that is Not assigned

In [47]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
        if toronto.iloc[i][2] == 'Not assigned':
            toronto.iloc[i][2] = toronto.iloc[i][1]
            i = i+1                               
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(15)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n, Malvern\n"
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n"
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"East Birchmount Park\n, Ionview\n, Kennedy Park\n"
7,M1L,Scarborough,"Clairlea\n, Golden Mile\n, Oakridge\n"
8,M1M,Scarborough,"Cliffcrest\n, Cliffside\n, Scarborough Village..."
9,M1N,Scarborough,"Birch Cliff\n, Cliffside West\n"


In [48]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n, Malvern\n"
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n"
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"East Birchmount Park\n, Ionview\n, Kennedy Park\n"
7,M1L,Scarborough,"Clairlea\n, Golden Mile\n, Oakridge\n"
8,M1M,Scarborough,"Cliffcrest\n, Cliffside\n, Scarborough Village..."
9,M1N,Scarborough,"Birch Cliff\n, Cliffside West\n"


In [49]:
df.shape

(103, 3)